In [27]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

from scipy.optimize import curve_fit
from scipy.stats import norm
import scipy.stats as stats

# ALTEZZA VETRINO

In [28]:
filename = '/data/reni_Z_cervello.csv'

data = pd.read_csv(dir_path + filename)
# data.head()

In [29]:
def gaussian(x, amp, mu, sigma, fondo):   # funzione gaussiana
    return amp * norm.pdf(x,loc = mu,scale = sigma) + fondo

In [30]:
def fittiamo(data):
    # Estrai i valori di x e y dai dati
    x = data['[micron]'].values
    y = data['Mean'].values

    # Stima iniziale dei parametri: amp, mu, sigma, fondo
    initial_guess = [max(y), x[np.argmax(y)], 1, min(y)]

    # Esegui il fit con curve_fit
    params, covariance = curve_fit(gaussian, x, y, p0=initial_guess)

    # Restituisci i parametri del fit e la matrice di covarianza
    return params, covariance

# Esempio di utilizzo della funzione
popt, pcov = fittiamo(data)
print("Parametri del fit:", params)

Parametri del fit: [1.19209211e+04 1.02259283e+01 4.10229949e+00 1.10743005e+02]


In [36]:
def r_assiale_teorica():
    lungh_onda = 632.8
    n = 1.47
    theta = 72
    return 1.4 * lungh_onda / (n * np.sin(theta) ** 2)

In [37]:
# Define x and y for the Gaussian fit
x = data["[micron]"].values
y = gaussian(x, *popt)  # Use the Gaussian function with the fitted parameters

fig = go.Figure()


# Aggiungi la curva gaussiana come linea
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines',
    name='Fit Gaussiano',
    line=dict(color='plum', width=2)
))

# Aggiungi i dati originali come scatter plot
fig.add_trace(go.Scatter(
    x=data["[micron]"],
    y=data["Mean"],
    mode='markers',
    name='Dati originali',
    marker=dict(color='purple', size=8)
))


# Configura il layout del grafico
fig.update_layout(
    xaxis_title="z (μm)",
    yaxis_title="Gray Value",
    plot_bgcolor="white",
    font=dict(size=18),
    xaxis=dict(showline=True, linecolor="black", gridcolor="lightgrey"),
    yaxis=dict(showline=True, linecolor="black", gridcolor="lightgrey")
)


fig.show()


In [38]:
# Calculate chi-squared value
residuals = data["Mean"].values - gaussian(data["[micron]"].values, *popt)
chi2 = np.sum((residuals ** 2) / data["Mean"].values)

# Calculate degrees of freedom
dof = len(data["Mean"]) - len(popt)

# Calculate p-value
p_value = 1 - stats.chi2.cdf(chi2, dof)

print(f"amp = {popt[0]:.2f} +- {np.sqrt(pcov[0,0]):.2f}")
print(f"mu = {popt[1]:.2f} +- {np.sqrt(pcov[1,1]):.2f}")
print(f"sigma = {popt[2]:.2f} +- {np.sqrt(pcov[2,2]):.2f}")
print(f"fondo = {popt[3]:.2f} +- {np.sqrt(pcov[3,3]):.2f}")
print(f"Chi2 = {chi2:.2f}")
print(f"p-value = {p_value:.2f}")

amp = 11920.92 +- 328.44
mu = 10.23 +- 0.03
sigma = 4.10 +- 0.07
fondo = 110.74 +- 16.23
Chi2 = 15.88
p-value = 0.98


In [39]:
from math import sqrt, log

# Extract sigma from the fitted parameters
sigma = popt[2]

# Calculate FWHM
fwhm = 2 * sqrt(2 * log(2)) * sigma

print(f"risoluzione assiale: FWHM = {fwhm:.2f}")

risoluzione assiale: FWHM = 9.66


In [40]:
print(r_assiale_teorica())

9354.357948463932
